#Tarea 1 Web Semantica y Datos Abiertos


#Conversión de datos a RDF


Conversión de formatos json y csv a rdf turtle mediante python
El código presentado a continuación se desarrolló especialmente para cumplir con lo propuesto en ésta taréa.

El código rdf resultante se validó en https://rdfshape.weso.es/dataInfo

##Enunciado
Seleccionar al menos dos conjuntos de datos abiertos que NO estén en formato RDF y transformar dichos datos en formato RDF. En el anexo se incluye una lista con algunas fuentes de datos abiertos.
**Opcional** : Seleccionar datos que en formato original estén en formatos diferentes, como puede ser: unos datos en CSV y otros en JSON, XML, etc.



### Fuente de datos 1

Datos organizacion administrativa de Chile

https://github.com/knxroot/bdcut-cl/tree/master

Datos recuperados en formato json

In [ ]:
!wget https://raw.githubusercontent.com/knxroot/bdcut-cl/master/BD/JSON/BDCUT_CL_ComunaProvinciaRegion__generado.json

--2023-08-01 19:59:41--  https://raw.githubusercontent.com/knxroot/bdcut-cl/master/BD/JSON/BDCUT_CL_ComunaProvinciaRegion__generado.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25786 (25K) [text/plain]
Saving to: ‘BDCUT_CL_ComunaProvinciaRegion__generado.json’

BDCUT_CL_ComunaProv 100%[===================>]  25.18K  --.-KB/s    in 0.001s  

2023-08-01 19:59:41 (25.3 MB/s) - ‘BDCUT_CL_ComunaProvinciaRegion__generado.json’ saved [25786/25786]



In [ ]:
!ls -l

total 32
-rw-r--r-- 1 root root 25786 Aug  1 19:59 BDCUT_CL_ComunaProvinciaRegion__generado.json
drwxr-xr-x 1 root root  4096 Jul 31 20:03 sample_data


In [ ]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
import json
import re
from rdflib import Literal

f = open("BDCUT_CL_ComunaProvinciaRegion__generado.json")
comunas_json=json.load(f)
print(comunas_json)

f.close()


[{'id': 1, 'name': 'Tarapacá', 'provincias': [{'id': 11, 'name': 'Iquique', 'comunas': [{'id': 1101, 'name': 'Iquique'}, {'id': 1107, 'name': 'Alto Hospicio'}]}, {'id': 14, 'name': 'Tamarugal', 'comunas': [{'id': 1401, 'name': 'Pozo Almonte'}, {'id': 1402, 'name': 'Camiña'}, {'id': 1403, 'name': 'Colchane'}, {'id': 1404, 'name': 'Huara'}, {'id': 1405, 'name': 'Pica'}]}]}, {'id': 2, 'name': 'Antofagasta', 'provincias': [{'id': 21, 'name': 'Antofagasta', 'comunas': [{'id': 2101, 'name': 'Antofagasta'}, {'id': 2102, 'name': 'Mejillones'}, {'id': 2103, 'name': 'Sierra Gorda'}, {'id': 2104, 'name': 'Taltal'}]}, {'id': 22, 'name': 'El Loa', 'comunas': [{'id': 2201, 'name': 'Calama'}, {'id': 2202, 'name': 'Ollagüe'}, {'id': 2203, 'name': 'San Pedro de Atacama'}]}, {'id': 23, 'name': 'Tocopilla', 'comunas': [{'id': 2301, 'name': 'Tocopilla'}, {'id': 2302, 'name': 'María Elena'}]}]}, {'id': 3, 'name': 'Atacama', 'provincias': [{'id': 31, 'name': 'Copiapó', 'comunas': [{'id': 3101, 'name': 'Copi

In [ ]:
# Covertir json a rdf turtle

# Caracters especiales serán reemplazados por "_" con
# re.sub(special_chars,"_",campo)



# Reemplaza cada caracter problemático por "_", para ser usado como nombre de un nodo
def rdf_name(s):
  special_chars="""[ "':.;]""" # Caracteres a reemplazar de strings de etiquetas
  res=re.sub(special_chars,"_",s)
  return res

ontologia="""prefix : <http://example.org/> .
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>
 :comuna   rdf:type    rdfs:Class .
 :provincia  rdf:type    rdfs:Class .
 :region     rdf:type    rdfs:Class .
"""



endline=" ;\n"
endgraph=" .\n"

# Variable en la que se almacenará el contenido en rdf turtle
comunas_rdf=ontologia+"\n"

type(comunas_rdf)


for region in comunas_json:
  comunas_rdf=f'{comunas_rdf}:region_{rdf_name(region["name"])} rdf:type  :region {endline}'
  comunas_rdf=f'{comunas_rdf}    schema:name    {Literal(region["name"]).n3()} {endline}'
  comunas_rdf=f'{comunas_rdf}    schema:id   "{region["id"]}" {endgraph}'
  for provincia in region['provincias']:
    comunas_rdf=f'{comunas_rdf}:provincia_{rdf_name(provincia["name"])} rdf:type  :provincia {endline}'
    comunas_rdf=f'{comunas_rdf}    schema:name    {Literal(provincia["name"]).n3()} {endline}'
    comunas_rdf=f'{comunas_rdf}    schema:id   "{provincia["id"]}" {endline}'
    comunas_rdf=f'{comunas_rdf}    schema:en_region  :region_{rdf_name(region["name"])} {endgraph}'
    for comuna in provincia['comunas']:
      comunas_rdf=f'{comunas_rdf}:comuna_{rdf_name(comuna["name"])} rdf:type  :comuna {endline}'
      comunas_rdf=f'{comunas_rdf}    schema:name    {Literal(comuna["name"]).n3()} {endline}'
      comunas_rdf=f'{comunas_rdf}    schema:id   "{comuna["id"]}" {endline}'
      comunas_rdf=f'{comunas_rdf}    schema:en_provincia  :provincia_{rdf_name(provincia["name"])} {endgraph}'

# print(comunas_rdf)



In [ ]:
print(comunas_rdf)

prefix : <http://example.org> .
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>
 :comuna   rdf:type    rdfs:Class .
 :provincia  rdf:type    rdfs:Class .
 :region     rdf:type    rdfs:Class .

:region_Tarapacá rdf:type  :region  ;
    schema:name    "Tarapacá"  ;
    schema:id   "1"  .
:provincia_Iquique rdf:type  :provincia  ;
    schema:name    "Iquique"  ;
    schema:id   "11"  ;
    schema:en_region  :region_Tarapacá  .
:comuna_Iquique rdf:type  :comuna  ;
    schema:name    "Iquique"  ;
    schema:id   "1101"  ;
    schema:en_provincia  :provincia_Iquique  .
:comuna_Alto_Hospicio rdf:type  :comuna  ;
    schema:name    "Alto Hospicio"  ;
    schema:id   "1107"  ;
    schema:en_provincia  :provincia_Iquique  .
:provincia_Tamarugal rdf:type  :provincia  ;
    schema:name    "Tamarugal"  ;
    schema:id   "14"  ;
    schema:en_region  :region_Tarapacá  .
:comuna_Pozo_Almonte rdf:type  :

In [ ]:
# Guardar rdf en archivo "comunas.rdf"
f=open("comunas.rdf","w")
f.write(comunas_rdf)
f.close()


###Fuente de datos 2

Observatorio logístico
Datos de red vial nacional por región
formato csv

In [ ]:
import random
import requests

def getApiKey():
    # Obtiene la APY_KEY requerida por la API publica de observatoriologistico.cl
    keyseed=1689910000000
    num=random.randint(0,9999999)
    keyseed=keyseed+num
    apikey_res=requests.get("https://datos.observatoriologistico.cl/manageDeveloper/create/?_="+str(keyseed))
    return apikey_res.json()["pApiKey"]

In [ ]:
apikey=getApiKey()

apikey

'hJCTQHaR2zKFXy9VhY8vfomZklLLiX8fS6Y96uZn'

In [ ]:
# Obtener datos Red vial Nacional

# API: https://api.datos.observatoriologistico.cl/api/v2/datastreams/RED-VIAL-NACIO-2015/data.csv/?auth_key=YOUR_API_KEY&limit=50


rvn_res=requests.get("https://api.datos.observatoriologistico.cl/api/v2/datastreams/RED-VIAL-NACIO-2015/data.csv/?auth_key="+apikey)


In [ ]:
import pandas as pd
from io import StringIO

# Convierte string the la respuesta a un objeto StringIO (para que pueda ser leido por dataframe.read_csv)
rvn_csv=StringIO(rvn_res.content.decode('utf-8'))

#Guardamos los datos obtenidos en un dataframe pandas
data = pd.read_csv(rvn_csv)

data.head(10)

print(data.iloc[3]["TIPO_CARPETA"])

Caminos Básicos Intermedios


In [ ]:
data

,REGIÓN,KILÓMETROS,TIPO_CARPETA,TIPO_RED,AÑO,NOMBRE_REGIÓN
0,I,"1.135,14",Asfalto,Red Vial Pavimentada,2015,Región de Tarapacá
1,I,"0,69",Hormigón,Red Vial Pavimentada,2015,Región de Tarapacá
2,I,"0,3",Asfalto/Hormigón,Red Vial Pavimentada,2015,Región de Tarapacá
3,I,0,Caminos Básicos Intermedios,Red Vial Pavimentada,2015,Región de Tarapacá
4,I,"319,22",Capa Protección,Soluciones Básicas,2015,Región de Tarapacá
...,...,...,...,...,...,...
739,XVI,"7,42",Caminos Básicos Intermedios,Red Vial Pavimentada,2020,Región de Ñuble
740,XVI,"342,92",Capa Protección,Soluciones Básicas,2020,Región de Ñuble
741,XVI,"17,46",Granular Estabilizado,Soluciones Básicas,2020,Región de Ñuble
742,XVI,"2.630,64",Ripio,Red Vial No Pavimentada,2020,Región de Ñuble


In [ ]:
ontologia="""prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>
:red_vial  rdf:type    rdfs:Class .
:region     rdf:type    rdfs:Class .

"""

endline=" ;\n"
endgraph=" .\n"

# Variable en la que se almacenará el contenido en rdf turtle
caminos_rdf=ontologia+"\n"

# Obtener grafos de regiones
reg_data=data[["REGIÓN","NOMBRE_REGIÓN"]].drop_duplicates()

for i in reg_data.index:
  caminos_rdf=f'{caminos_rdf}:{rdf_name(reg_data["NOMBRE_REGIÓN"][i])} rdf:type  :region {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:name    {Literal(reg_data["NOMBRE_REGIÓN"][i]).n3()} {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:num_region   {Literal(reg_data["REGIÓN"][i]).n3()} {endgraph}'


In [ ]:
# Obtener grafos de red_vial
for i in data.index:
  caminos_rdf=f'{caminos_rdf}:{i} rdf:type  :red_vial {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:en_region    :{rdf_name(data["NOMBRE_REGIÓN"][i])} {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:tipo_carpeta   {Literal(data["TIPO_CARPETA"][i]).n3()} {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:tipo_red   {Literal(data["TIPO_RED"][i]).n3()} {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:kilometros   {Literal(data["KILÓMETROS"][i]).n3()} {endline}'
  caminos_rdf=f'{caminos_rdf}    schema:año   {Literal(data["AÑO"][i]).n3()} {endgraph}'

print(caminos_rdf)


prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>
:red_vial  rdf:type    rdfs:Class .
:region     rdf:type    rdfs:Class .


:Región_de_Tarapacá rdf:type  :region  ;
    schema:name    "Región de Tarapacá"  ;
    schema:num_region   "I"  .
:Región_de_Antofagasta rdf:type  :region  ;
    schema:name    "Región de Antofagasta"  ;
    schema:num_region   "II"  .
:Región_de_Atacama rdf:type  :region  ;
    schema:name    "Región de Atacama"  ;
    schema:num_region   "III"  .
:Región_de_Coquimbo rdf:type  :region  ;
    schema:name    "Región de Coquimbo"  ;
    schema:num_region   "IV"  .
:Región_de_Valparaíso rdf:type  :region  ;
    schema:name    "Región de Valparaíso"  ;
    schema:num_region   "V"  .
:Región_del_Libertador_General_Bernardo_O_Higgins rdf:type  :region  ;
    schema:name    "Región del Libertador General Bernardo O'Higgins"  ;
    schema:num_re

In [ ]:
# Guardar rdf en archivo "red_vial.rdf"
f=open("red_vial.rdf","w")
f.write(caminos_rdf)
f.close()
